In [23]:
import redis, json
import pandas as pd
import logging_to_cloudwatch as ltc
import utils

In [24]:
with open("../../.KEYS/DATA_SRC_INFO.json", "r") as f:
    storage_info = json.load(f)

In [25]:
redis_ip = storage_info['redis_conn_info']['ip']
redis_port = storage_info['redis_conn_info']['port']

redis_session = redis.StrictRedis(host=redis_ip, port=redis_port, db=0)

In [26]:
id_list = pd.read_csv("selected.csv").pid.to_list()
id_records = [{"id": id, "get_date": "20240924"} for id in id_list]

In [27]:
def check_id_in_redis_with_transaction(logger, redis_session, records):
    inited_id_records = []
    with redis_session.pipeline() as pipe:
        try:
            logger.info("watch compare_id redis hash-key")
            redis_session.watch('compare_id')
            logger.info("set redis read transaction.")
            pipe.multi()
            for record in records:
                _id = record.get('id')
                pipe.hexists('compare_id', _id)
            logger.info("start redis read transaction execute.")
            exists = pipe.execute()
            for exist, record in zip(exists, records):
                if not exist:
                    inited_id_records.append(record)
                    logger.info(f"id record {record} is not exist in redis. Added to upload list.")
                else:
                    logger.info(f"id {record} already exists in redis. This id passed.")
        except redis.WatchError: # 만약 다른 클라이언트가 키를 수정하여 트랜잭션이 실패한 경우
            logger.warning("Transaction failed. Retrying the check for ids...")
            return check_id_in_redis_with_transaction(logger, redis_session, records)  # 재시도
        except Exception as e:
            logger.error(f"An error occurred during the Redis read transaction: {e}")
        finally:
            logger.info("unwatch compare_id redis hash-key")
            redis_session.unwatch()
            logger.info(f"Completed checking ids: target ids are {inited_id_records}")
            
            return inited_id_records

In [28]:
# record에서 없는 id만 redis로 push
def upload_id_into_redis(logger, redis_session, records):
    logger.info("start upload id's into redis")
    with redis_session.pipe() as pipe:
        try:
            logger.info("watch compare_id redis hash-key")
            redis_session.watch('compare_id')
            logger.info("set redis read transaction.")
            pipe.multi()
            for record in records:
                _id = record.get('id')
                _get_date = record.get('get_date')
                pipe.hset('compare_id', _id, _get_date)
            logger.info("start redis upload transaction execute.")
            pipe.execute()
        except Exception as e:
            logger.error(f"An error occurred during the Redis upload transaction: {e}")
        finally:
            logger.info("unwatch compare_id redis hash-key")
            redis_session.unwatch()
    logger.info(f"Completed upload id's in redis.")

In [29]:
logger = ltc.log('/aws/preprocessing/wanted-first','wanted_logs')
result = check_id_in_redis_with_transaction(logger, redis_session, id_records)

In [30]:
result

[]